In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import tqdm 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import os


In [4]:
train_labels0=[]
test_labels0=[]
train_imgs0=[]
test_imgs0=[]
classes=['Bear', 'Bull', 'Cat', 'Cattle',
           'Cheetah', 'Chicken', 'Crocodile',
           'Deer', 'Dog', 'Fox', 'Goat', 'Hippopotamus', 
           'Horse', 'Jaguar', 'Leopard', 'Lion', 'Lynx',
           'Monkey', 'Mule', 'Ostrich', 'Owl', 
           'Pig', 'Raccoon', 'Raven', 'Rhinoceros', 
           'Sheep', 'Snake', 'Tiger', 'Turkey', 'Turtle', 'Zebra']
"""classes=['Cat','Dog']"""
encode={}
for i in range(len(classes)):
    encode[classes[i]]=i+1

def checkclass(dirname):
    
    for animal in classes:
        if dirname.endswith(animal):
            return True
    return False

for dirname, dirs, filenames in os.walk('./data/'):
    
    if 'train' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        
        for label in sorted(os.listdir(labels)):
            
            
            if label.endswith('txt'):
                train_labels0.append(labels+label)
        
        for image in sorted(filenames):
            
            
            if image.endswith('png') or image.endswith('jpg'):
                train_imgs0.append(dirname+'/'+image)
    if 'val' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        for label in sorted(os.listdir(labels)):
            if label.endswith('.txt'):
                test_labels0.append(labels+label)
        for image in sorted(filenames):
            if image.endswith('png')or image.endswith('jpg'):
                test_imgs0.append(dirname+'/'+image)


In [44]:
print(len(train_imgs0))
print(len(train_labels0))
print(len(test_imgs0))
print(len(test_labels0))
train_labels=[]
test_labels=[]
train_imgs=[]
test_imgs=[]
val_imgs=[]
val_imgs=[]
for i in range(len(train_labels0)):
    if i%4==0:
        
        train_labels.append(train_labels0[i])
        train_imgs.append(train_imgs0[i])
for i in range(len(test_labels0)):
        test_labels.append(test_labels0[i])
        test_imgs.append(test_imgs0[i])
print(len(train_imgs))
print(len(train_labels))
print(len(test_imgs))
print(len(test_labels))


14620
14620
6864
6864
3655
3655
6864
6864


In [6]:
class AnimalDataset(object):
    def __init__(self, transforms,img_dir,label_dir):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = img_dir
        self.label=label_dir
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = self.imgs[idx]
        file_label = self.label[idx]
        img = Image.open(file_image).convert("RGB")
        target={}
        target['image_id']=torch.tensor([idx])
        labels=[]
        boxes=[]
        #Generate Label
        with open(file_label,'r')as f:
            lines=f.readlines()
            for line in lines:
                line_list=line.strip().split(' ')
                labels.append(encode[line_list[0]])
                boxes.append([float(i) for i in line_list[1:]])
                
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)        
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        iscrowd = torch.zeros((len(classes),), dtype=torch.int64)

        target['boxes']=boxes
        target['labels']=labels
       
        target["area"] = area
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [7]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [8]:
import torchvision
from torchvision.models.detection.ssd import SSD
from torchvision.models.detection.rpn import AnchorGenerator
def get_model_instance_vgg19(num_classes):
    backbone = torchvision.models.vgg19().features
    
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))
    size=(300,300)
    model=SSD(backbone=backbone,anchor_generator=anchor_generator,size=size,num_classes=num_classes)
    return model

In [9]:
model=get_model_instance_vgg19(len(classes)+1)

In [7]:
from engine import train_one_epoch, evaluate
import utils
import time
train_data=AnimalDataset(get_transform(train=False),train_imgs,train_labels)
val_data=AnimalDataset(get_transform(train=False),train_imgs[:100],train_labels[:100])
test_data=AnimalDataset(get_transform(train=False),test_imgs[:100],test_labels[:100])
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person


# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=4, shuffle=True, num_workers=0,
    collate_fn=utils.collate_fn)
val_loader = torch.utils.data.DataLoader(
    val_data, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

# get the model using our helper function

model =get_model_instance_vgg19(len(classes)+1)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=10,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 50
starttime=time.time()

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=100)
    # update the learning rate
    torch.save(model.state_dict(), "./model/SSD/SSD_vgg19"+str(epoch)+".pt")
    
    print('saved dict')
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model,val_loader, device=device)
    evaluate(model,test_loader, device=device)
print("training time: "+str(time.time()-starttime)+"s")

0it [00:00, ?it/s]

2it [00:02,  1.06s/it]

Epoch: [0]  [   0/2437]  eta: 1:33:05  lr: 0.000010  loss: 30.1254 (30.1254)  bbox_regression: 5.1153 (5.1153)  classification: 25.0101 (25.0101)  time: 2.2920  data: 0.0825  max mem: 791


102it [00:20,  5.78it/s]

Epoch: [0]  [ 100/2437]  eta: 0:07:48  lr: 0.000509  loss: 6.7575 (14.5556)  bbox_regression: 2.0530 (2.6525)  classification: 4.8007 (11.9031)  time: 0.1742  data: 0.0761  max mem: 932


...

Epoch: [49]  [2300/2437]  eta: 0:00:19  lr: 0.000001  loss: 0.5381 (0.6518)  bbox_regression: 0.1297 (0.1513)  classification: 0.4062 (0.5005)  time: 0.1363  data: 0.0407  max mem: 1250


2402it [05:33,  7.31it/s]

Epoch: [49]  [2400/2437]  eta: 0:00:05  lr: 0.000001  loss: 0.6540 (0.6510)  bbox_regression: 0.1262 (0.1509)  classification: 0.5088 (0.5001)  time: 0.1344  data: 0.0389  max mem: 1250


2437it [05:38,  7.20it/s]


Epoch: [49]  [2436/2437]  eta: 0:00:00  lr: 0.000001  loss: 0.5830 (0.6504)  bbox_regression: 0.1344 (0.1507)  classification: 0.4212 (0.4997)  time: 0.1357  data: 0.0398  max mem: 1250
flag
Epoch: [49] Total time: 0:05:38 (0.1388 s / it)
saved dict
creating index...
index created!
Test:  [  0/100]  eta: 0:00:03  model_time: 0.0200 (0.0200)  evaluator_time: 0.0000 (0.0000)  time: 0.0330  data: 0.0120  max mem: 1250
Test:  [ 99/100]  eta: 0:00:00  model_time: 0.0280 (0.0261)  evaluator_time: 0.0010 (0.0006)  time: 0.0424  data: 0.0120  max mem: 1250
flag
Test: Total time: 0:00:03 (0.0399 s / it)
Averaged stats: model_time: 0.0280 (0.0261)  evaluator_time: 0.0010 (0.0006)
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.764
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.929
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.926
 Average Prec

In [17]:
import time

from PIL import Image, ImageDraw,ImageFont
from engine import train_one_epoch, evaluate
import utils
# Load the model\
model=get_model_instance_vgg19(len(classes)+1)
model.load_state_dict(torch.load('./model/SSD/SSD_vgg1920.pt'))
model.to('cuda')
model.eval()
test_data=AnimalDataset(get_transform(train=False),train_imgs[6969:6970],train_labels[6969:6970])
# Load a batch of images
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

timecount=0
for imgs,target in test_loader:
    print('hi')
    images = list(img.to('cuda') for img in imgs)
    torch.cuda.synchronize()
    start_time = time.time()
    # Run the model on the images
    with torch.no_grad():
        
        outputs = model(images)
    timecount+=time.time()-start_time
    # Measure the end time
    boxes = outputs[0]['boxes']

    # Load the image using PIL
    

    # Draw the boxes on the image
    
        

    # Display the image
    img = Image.open(train_imgs[6969])

    # Create a drawing object
    draw = ImageDraw.Draw(img)

    # Define the font to use for the labels
    font = ImageFont.truetype("/usr/share/fonts/truetype/abyssinica/AbyssinicaSIL-Regular.ttf", 16)

    # Define the bounding boxes and labels
    
    labels = [classes[label]for label in outputs[0]['labels']]
    print(labels)
    print(boxes)
    # Draw the bounding boxes and labels
    for box, label in zip(boxes, labels):
        draw.rectangle(box.tolist(), outline="red", width=2)
        draw.text((box), label, font=font)

    # Display the image
    img.show()
# Measure the start time


# Calculate the inference time per image

In [54]:
import time
from PIL import Image, ImageDraw,ImageFont
i=3415
model=get_model_instance_vgg19(len(classes)+1)
model.load_state_dict(torch.load('./model/SSD/SSD_vgg1920.pt'))
model.to('cuda')
model.eval()
test_data=AnimalDataset(get_transform(train=False),train_imgs[i:i+1],train_labels[i:i+1])
# Load a batch of images
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

timecount=0
for imgs,target in test_loader:
    images = list(img.to('cuda') for img in imgs)
    torch.cuda.synchronize()
    start_time = time.time()
    # Run the model on the images
    with torch.no_grad():
        
        outputs = model(images)
    timecount+=time.time()-start_time
    # Measure the end time
    print(outputs)
    max_pred=torch.argmax(outputs[0]['scores'])
    boxes = [outputs[0]['boxes'][max_pred]   ]  
    # Load the image using PIL
    

    # Draw the boxes on the image
    
        

    # Display the image
    img = Image.open(train_imgs[i])

    # Create a drawing object
    draw = ImageDraw.Draw(img)

    # Define the font to use for the labels
    font = ImageFont.truetype("/usr/share/fonts/truetype/abyssinica/AbyssinicaSIL-Regular.ttf", 16)

    # Define the bounding boxes and labels
    
    labels = [classes[outputs[0]['labels'][max_pred]-1]]
    print(labels)
    print(boxes)
    # Draw the bounding boxes and labels
    for box, label in zip(boxes, labels):
        draw.rectangle(box.tolist(), outline="red", width=2)
        draw.text((box), label, font=font)

    # Display the image
    img.show()
# Measure the start time



[{'boxes': tensor([[206.8491, 411.0158, 611.4329, 930.5374],
        [206.8491, 411.0158, 611.4329, 930.5374],
        [206.8491, 411.0158, 611.4329, 930.5374]], device='cuda:0'), 'scores': tensor([0.8746, 0.0252, 0.0106], device='cuda:0'), 'labels': tensor([18,  9, 23], device='cuda:0')}]
['Monkey']
[tensor([206.8491, 411.0158, 611.4329, 930.5374], device='cuda:0')]
